In [1]:
!pip install ultralytics
!pip install boto3

In [1]:
from ultralytics import YOLO
import boto3
import os
import torch

print(torch.cuda.is_available()) # comment this line if no gpu available

True


In [3]:
def download_data_from_s3(client, bucket_name, prefix, local_dir):
    if not os.path.exists(local_dir):
        os.makedirs(local_dir)
    for obj in client.list_objects_v2(Bucket=bucket_name, Prefix=prefix)['Contents']:
        if not os.path.exists(os.path.dirname(obj['Key'])):
            os.makedirs(os.path.dirname(obj['Key']))
        client.download_file(bucket_name, obj['Key'], obj['Key'])

def get_keys(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()
        if len(lines) != 2:
            raise Exception("Unable to read keys. Invalid file format.")
        return (lines[0][:-1], lines[1])

id, secret = get_keys("aws_keys.txt")
download_data_from_s3(boto3.client('s3', aws_access_key_id=id, aws_secret_access_key=secret), "sagemaker-us-east-1-992382414571", "datasets", "datasets")

In [2]:
model = YOLO('yolov8n.pt')
model.to('cuda') # comment this line if no gpu available

results = model.train(data="datasets/geoguessr.yaml", epochs=150, imgsz=1024, pretrained=True, fraction=0.8, iou=0.7, visualize=True, patience=0)
results = model.val()

New https://pypi.org/project/ultralytics/8.2.73 available  Update with 'pip install -U ultralytics'
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=datasets/geoguessr.yaml, epochs=150, time=None, patience=0, batch=16, imgsz=1024, save=True, save_period=-1, cache=False, device=cuda:0, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=0.8, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=True, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=

train: Scanning C:\programming\GeoGuessr-AI\model\datasets\bollards\labels\train.cache... 37 images, 0 backgrounds, 0 corrupt: 100%|██████████| 37/37 [00:00<?, ?it/s]
val: Scanning C:\programming\GeoGuessr-AI\model\datasets\bollards\labels\test.cache... 4 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4/4 [00:00<?, ?it/s]


Plotting labels to runs\detect\train\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 1024 train, 1024 val
Using 8 dataloader workers
Logging results to runs\detect\train
Starting training for 150 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/150      5.51G      1.507       7.24      1.078          7       1024: 100%|██████████| 3/3 [00:03<00:00,  1.31s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  3.91it/s]

                   all          4          5     0.0025        0.6     0.0511     0.0355


...


150 epochs completed in 0.040 hours.
Optimizer stripped from runs\detect\train\weights\last.pt, 6.3MB
Optimizer stripped from runs\detect\train\weights\best.pt, 6.3MB

Validating runs\detect\train\weights\best.pt...
Ultralytics YOLOv8.2.31  Python-3.12.4 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 16380MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00, 12.47it/s]


                   all          4          5          1      0.989      0.995      0.721
Speed: 0.2ms preprocess, 2.8ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to runs\detect\train
Ultralytics YOLOv8.2.31  Python-3.12.4 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 16380MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning C:\programming\GeoGuessr-AI\model\datasets\bollards\labels\test.cache... 4 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4/4 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:05<00:00,  5.05s/it]


                   all          4          5          1      0.989      0.995      0.658
Speed: 2.6ms preprocess, 47.4ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs\detect\train2


In [3]:
model.export()

Ultralytics YOLOv8.2.31  Python-3.12.4 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 16380MiB)

PyTorch: starting from 'runs\detect\train\weights\best.pt' with input shape (1, 3, 1024, 1024) BCHW and output shape(s) (1, 5, 21504) (6.0 MB)

TorchScript: starting export with torch 2.3.1+cu121...
TorchScript: export success  1.3s, saved as 'runs\detect\train\weights\best.torchscript' (12.0 MB)

Export complete (1.5s)
Results saved to C:\programming\GeoGuessr-AI\model\runs\detect\train\weights
Predict:         yolo predict task=detect model=runs\detect\train\weights\best.torchscript imgsz=1024  
Validate:        yolo val task=detect model=runs\detect\train\weights\best.torchscript imgsz=1024 data=datasets/geoguessr.yaml  
Visualize:       https://netron.app


'runs\\detect\\train\\weights\\best.torchscript'